# Car System Agentic AI - Template

**Project:** Agentic AI example  
**Author:** Klaus  
**License:** MIT License

---

This notebook demonstrates the car diagnostic system using LangGraph with multiple specialized nodes for input validation, reasoning, and output processing.


# Imports

In [1]:
exec(open('setup.py').read())

from notebooks import *
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from src.data_models.graph_state import CarSystemState
from langchain_core.runnables import RunnableConfig
from langgraph.types import Command
from src.tools.car import get_car_status
from src.tools.calculations import is_trip_possible
from src.utils.prompt_loader import get_prompt


[2025-09-19 17:33:50] (__init__.py:46) INFO | 🔑 Environment loaded from: /home/klaus/Projects/car-system-agentic-ai/.env
[2025-09-19 17:33:50] (__init__.py:48) INFO | 🔑 GOOGLE_API_KEY: ✅ Found


# Load Prompts

In [2]:
# Load all prompts
output_guard_rail_prompt = get_prompt("output_guard_rail")
input_guard_rail_prompt = get_prompt("input_guard_rail") 
reasoning_node_prompt = get_prompt("reasoning_node")

[2025-09-19 17:33:50] (prompt_loader.py:129) INFO | 🔍 PromptLoader: Loaded prompt output_guard_rail
[2025-09-19 17:33:50] (prompt_loader.py:129) INFO | 🔍 PromptLoader: Loaded prompt input_guard_rail
[2025-09-19 17:33:50] (prompt_loader.py:129) INFO | 🔍 PromptLoader: Loaded prompt reasoning_node


# Definitions

## Agents

In [3]:
# model definition with prompts from markdown files
input_guard_rail_agent = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=input_guard_rail_prompt
)

reasoning_agent = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=reasoning_node_prompt
)

output_guard_rail_agent = models.GeminiModel(
    model="gemini-2.5-flash", 
    prompt=output_guard_rail_prompt
)

[2025-09-19 17:33:50] (gemini.py:38) INFO | 🤖 GeminiModel: Initialized with model gemini-2.5-flash
[2025-09-19 17:33:50] (gemini.py:38) INFO | 🤖 GeminiModel: Initialized with model gemini-2.5-flash
[2025-09-19 17:33:50] (gemini.py:38) INFO | 🤖 GeminiModel: Initialized with model gemini-2.5-flash


## Nodes

In [4]:
# node definition
ENTRYPOINT = START
INPUT_GUARD_RAIL = "input_guard_rail"
REASONING_NODE = "reasoning_node"
OUTPUT_GUARD_RAIL = "output_guard_rail"
EXIT_ZONE = "end"

input_guard_rail = nodes.InputGuardRail(
    routing_options={"next_node": REASONING_NODE, "end": OUTPUT_GUARD_RAIL},
    model=input_guard_rail_agent,
)

reasoning_node = nodes.NodeWithTools(
    routing_options={"next_node": OUTPUT_GUARD_RAIL, "end": OUTPUT_GUARD_RAIL},
    model=reasoning_agent,
    tools=[get_car_status, is_trip_possible],
)

output_guard_rail = nodes.OutputGuardRail(
    routing_options={"end": EXIT_ZONE},
    model=output_guard_rail_agent,
)

[2025-09-19 17:33:50] (input_guard_rail.py:36) INFO | InputGuardRail: Initialized
[2025-09-19 17:33:50] (node_with_tools.py:55) INFO | NodeWithTools: Initialized
[2025-09-19 17:33:50] (output_guard_rail.py:41) INFO | OutputGuardRail: Initialized


## Graph Creation

In [5]:
# Create the graph with state_schema parameter
workflow = StateGraph(state_schema=CarSystemState)

# Add nodes to the graph
workflow.add_node(INPUT_GUARD_RAIL, input_guard_rail)
workflow.add_node(REASONING_NODE, reasoning_node)
workflow.add_node(OUTPUT_GUARD_RAIL, output_guard_rail)

# Define the flow with conditional routing
# Os nodes usam Command para determinar o próximo node baseado no estado
workflow.add_edge(START, "input_guard_rail")
#workflow.add_edge(EXIT_ZONE, lambda state: Command(goto=END))

app = workflow.compile()

# Test Graph Execution

In [6]:
# Alternative: Synchronous test (if async has issues)
def test_graph_sync():
    """Synchronous version of the graph test."""
    
    # Create a test state using CarSystemState
    test_input = CarSystemState(
        messages=[HumanMessage(content="Quero viajar 300 km amanhã. Com o combustível que tenho dá para ir sem reabastecer?")],
        user_input=None,
        car_data=None,
        current_node=None,
        processing_status=None,
        analysis_result=None,
        recommendations=None,
        error_message=None,
        context=None,
    )

    config = RunnableConfig(config={"run_name": "car-system-agentic-ai"})

    
    print("🔧 Testing graph execution (synchronous)...")
    print(f"Input message: {test_input['messages'][0].content}")
    print("-" * 60)
    
    try:
        # Run the graph synchronously
        result = app.invoke(input=test_input, config=config)
        print("-" * 60)
        print("✅ Synchronous graph execution successful!")
        print("\n📊 Final State:")
        print("-" * 30)
        
        # Display the results
        for key, value in result.items():
            if value is not None:
                if key == "messages":
                    print(f"📝 {key}: {len(value)} messages")
                    for i, msg in enumerate(value):
                        print(f"   [{i}] {type(msg).__name__}: {msg.content[:80]}...")
                elif isinstance(value, str) and len(value) > 80:
                    print(f"📄 {key}: {value[:80]}...")
                else:
                    print(f"🔹 {key}: {value}")
        
        return result
        
    except Exception as e:
        print(f"❌ Error during synchronous execution: {e}")
        print(f"Error type: {type(e).__name__}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")
        return None

print("\n" + "="*60)
print("🔄 Running synchronous version...")
sync_result = test_graph_sync()

if sync_result:
    print(f"\n🎉 Sync test completed! Final state has {len([k for k, v in sync_result.items() if v is not None])} populated fields.")
else:
    print("\n❌ Sync test also failed. Check configuration.")
print("\n" + "="*60)


🔄 Running synchronous version...
🔧 Testing graph execution (synchronous)...
Input message: Quero viajar 300 km amanhã. Com o combustível que tenho dá para ir sem reabastecer?
------------------------------------------------------------
[2025-09-19 17:33:50] (input_guard_rail.py:44) INFO | 🛡️ InputGuardRail: Starting execution
[2025-09-19 17:33:50] (input_guard_rail.py:59) INFO | 📨 Processing message: Quero viajar 300 km amanhã. Com o combustível que tenho dá para ir sem reabastecer?...
[2025-09-19 17:33:52] (gemini.py:74) INFO | 🪙 Token usage: {'input_tokens': 385, 'output_tokens': 112, 'total_tokens': 497, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 95}}
[2025-09-19 17:33:52] (input_guard_rail.py:71) INFO | ✅ Validation result: is_valid=True
[2025-09-19 17:33:52] (input_guard_rail.py:77) INFO | 🚀 Routing to next_node: reasoning_node
[2025-09-19 17:33:52] (node_with_tools.py:118) INFO | 🧠 NodeWithTools: Starting execution
[2025-09-19 17:33:52] (node